In [1]:
from coffea import hist
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import mplhep as hep
import numpy as np
import random
plt.style.use(hep.style.CMS)

import awkward as ak

from matplotlib import colors
POPTS={'norm':colors.LogNorm()}

In [2]:
from data import getData, repackage
ldmx_dict = getData(chunks=True, fnames="/Users/chloeg/Desktop/Work/Fermilab2021/HistData/kshort_pn_4GeV_Jun7_ntuple/*.root")

In [3]:
def extend_array(arr, new_attr, new_attr_name):
    members={n:arr[n] for n in arr.fields}
    members[new_attr_name] = new_attr
    return ak.zip(members)

def add_angle(arr,br=['px','py','pz','e'],name="theta"):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    arr = extend_array(arr, part.theta, name)
    return arr

def get_vector(arr):
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)

    part =  ak.zip({"x": arr.px,
                    "y": arr.py,
                    "z": arr.pz,
                    "t": arr.e,
                    },
                    with_name="LorentzVector")
    return part

def flat(x,axis=None): # for now must cast while waiting for coffea to catch up
    try:
        return ak.to_numpy(ak.flatten(x,axis=axis)) 
    except:
        return x



In [4]:
def track_smear(kinetic_energy, theta):
    c_eres = random.gauss(1.,0.06);
    c_hres = random.gauss(0.,0.02);
    return (c_eres*kinetic_energy, abs(c_hres+theta))

def piecewise_smear(pz):
    c_eres_array = []
    for part in pz:
        for element in part:
            if element <= 50:
                c_eres = random.gauss(1,0.0404);
            elif element <= 100: 
                c_eres = random.gauss(1,0.0410);
            elif element <= 250: 
                c_eres = random.gauss(1,0.0415);
            elif element <= 500: 
                c_eres = random.gauss(1,0.0420);
            elif element <= 1000: 
                c_eres = random.gauss(1,0.0435);
            elif element <= 2000:  
                c_eres = random.gauss(1,0.0477);
            elif element <= 3000:  
                c_eres = random.gauss(1,0.0523);
            elif element <= 4000:  
                c_eres = random.gauss(1,0.0573);            
            else:    
                c_eres = random.gauss(1,0.06);
            c_eres_array=np.append(c_eres_array, [c_eres])
    return (c_eres_array)

In [5]:
hists = {}
hists["Target_pion_ke"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"Kinetic Energy [MeV]", 30, 0, 4000),
                                hist.Bin("keS", r"Energy [MeV]", 30, 0, 4000),
                            )       
hists["Target_pion_theta"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"$\theta [rad]$", 30, 0, 2),
                                hist.Bin("keS", r"$\theta [rad]$", 30, 0, 2),
                            ) 
hists["Target_pion_mass"] = hist.Hist("Target SP Kaons",    
                                hist.Bin("keOG", r"Reconstructed $K_s$ Mass [MeV]", 80, 0, 1000),
                                hist.Bin("keS", r"Reconstructed $K_s$ Mass [MeV]", 80, 0, 1000),
                            )

In [6]:
def ProcessChunk(chunk, hists):
    ldmx_events = repackage(chunk)

    target_kaon_dau1 = ldmx_events['TSP_Kaon_dau1']
    target_kaon_dau2 = ldmx_events['TSP_Kaon_dau2']
    
    kshort_pdgid = 310
    chargedpion_pdgid = 211
    pion_mass = 139
    kshort_mass = 497.648
    
    mask_2dau1 = (target_kaon_dau2.gd==0)
    mask_2dau2 = (target_kaon_dau2.gd==0)
    kshort_mask = (abs(target_kaon_dau2.mompdgID)==kshort_pdgid)
    pidau1_mask = (abs(target_kaon_dau2.pdgID)==chargedpion_pdgid)
    pidau2_mask = (abs(target_kaon_dau2.pdgID)==chargedpion_pdgid)
    
    lower_mask1 = ((-1*((target_kaon_dau2.mome) - kshort_mass)+2000) <= (target_kaon_dau2.electron_e))
    lower_mask2 = ((-1*((target_kaon_dau2.mome) - kshort_mass)+2000) <= (target_kaon_dau2.electron_e))
      
    lower_mask3 = lower_mask1[pidau1_mask & kshort_mask & mask_2dau1]  
    
    pi1 = get_vector(target_kaon_dau1[pidau1_mask & kshort_mask & mask_2dau1])
    pi2 = get_vector(target_kaon_dau2[pidau2_mask & kshort_mask & mask_2dau2])
    
    new_ke1 = track_smear(pi1.t - pion_mass, pi1.theta)[0]
    new_ke2 = track_smear(pi2.t - pion_mass, pi2.theta)[0]
    new_theta1 = track_smear(pi1.t - pion_mass, pi1.theta)[1]
    new_theta2 = track_smear(pi2.t - pion_mass, pi2.theta)[1]
    new_pz1 = piecewise_smear(pi1.pz)*flat(pi1.pz)
    new_psq1 = flat(pi1.px)**2+flat(pi1.py)**2+new_pz1**2
    new_pz2 = piecewise_smear(pi2.pz)*flat(pi2.pz)
    new_psq2 = flat(pi2.px)**2+flat(pi2.py)**2+new_pz2**2
    
    from coffea.nanoevents.methods import vector
    ak.behavior.update(vector.behavior)
    
    pisum = (pi1+pi2)
    
    #print(flat(pi1).shape)
    #print(flat(pi2).shape)
    
    pi1_track_smeared =  ak.zip({"x": flat(pi1.px),
                    "y": flat(pi1.py),
                    "z": new_pz1,
                    "t": (np.sqrt(new_psq1+flat(pi1.mass)**2)),
                    },
                    with_name="LorentzVector")
    pi2_track_smeared =  ak.zip({"x": flat(pi2.px),
                    "y": flat(pi2.py),
                    "z": new_pz2,
                    "t": (np.sqrt(new_psq2+flat(pi2.mass)**2)),
                    },
                    with_name="LorentzVector")
    pi_smeared_sum = (pi1_track_smeared+pi2_track_smeared)
  
    masks_id = {'Kshort': 310,
                'Klong': 130,
                'Kpm': 321,
               }
    masses = {'Kshort': 497.648,
              'Klong': 497.648,
              'Kpm': 493.677
             }
    #print(flat(pi_smeared_sum.t).shape)
    #print(flat(pi_smeared_sum[lower_mask3].t).shape)
    
    hists["Target_pion_ke"].fill(
                                 keOG=flat(pisum[lower_mask3].t-kshort_mass),
                                 keS=flat(pi_smeared_sum[lower_mask3].t-kshort_mass),
                                )
    hists["Target_pion_theta"].fill(
                                 keOG=flat(pisum[lower_mask3].theta),
                                 keS=flat(pi_smeared_sum[lower_mask3].theta),
                                )
    hists["Target_pion_mass"].fill(
                                 keOG=flat(pisum[lower_mask3].mass),
                                 keS=flat(pi_smeared_sum[lower_mask3].mass),
                                )
    
    return

In [7]:
nchunk = 0
for chunk in ldmx_dict:
    #if nchunk >= 1: break
    nchunk += 1
    print('process',nchunk)
    ProcessChunk(chunk, hists)

KeyInFileError: not found: 'Sim_Kaon_electrone'
in file /Users/chloeg/Desktop/Work/Fermilab2021/HistData/kshort_pn_4GeV_Jun7_ntuple/pn_kshortfilter_1M_events_r0048_b59540.root
in object /Events;1

In [ ]:
print(hists["Target_pion_ke"])
fig, ax = plt.subplots(1,3,figsize=(24,6), constrained_layout=True)
hist.plot1d(hists["Target_pion_ke"].sum('keS'),ax=ax[0],clear=False);
hist.plot1d(hists["Target_pion_ke"].sum('keOG'),ax=ax[0],clear=False);
leg = ax[0].legend([r'Measured $\pi ^\pm$', r'Smeared $\pi ^\pm$'])

hist.plot1d(hists["Target_pion_theta"].sum('keS'),ax=ax[1],clear=False);
hist.plot1d(hists["Target_pion_theta"].sum('keOG'),ax=ax[1],clear=False);
leg = ax[1].legend([r'Measured $\pi ^\pm$', r'Smeared $\pi ^\pm$'])

hist.plot1d(hists["Target_pion_mass"].sum('keS'),ax=ax[2],clear=False);
hist.plot1d(hists["Target_pion_mass"].sum('keOG'),ax=ax[2],clear=False);
leg = ax[2].legend([r'Measured $\pi ^\pm$', r'Smeared $\pi ^\pm$'])